# Working with VIEWS in MySQL

## Base Table

PreRequisite:  
The table `emp` exists and contains employee data such as:

- empno
- ename
- sal
- deptno

In [ ]:
SELECT * FROM emp;

## Simple View with Filtering

HR frequently queries employees belonging to **Department 20**. Instead of rewriting the filter each time, a reusable object is required.

A **VIEW** stores a SELECT query definition.
It behaves like a virtual table and always reflects current data from the base table.

In [ ]:
CREATE OR REPLACE VIEW deptno20 AS
SELECT *
FROM emp
WHERE deptno = 20;

In [ ]:
SELECT * FROM deptno20;

## DML Through a View (Insert)

HR wants to insert new employees directly into the department-specific view.

### Concept
In MySQL, a view is **updatable** if:
- It is based on a single table
- It does not use GROUP BY, DISTINCT, aggregates, or joins

DML executed on such a view affects the **base table**.

In [ ]:
INSERT INTO deptno20 VALUES 
(9000,'ARJUN','NAYAK',7782,'1982-01-23',1300.00,NULL,20);

In [ ]:
SELECT * FROM deptno20;

## 3. Aggregate View

### Business Use Case
Management wants to see **employee count per department**.

### Concept
Views that use:
- GROUP BY
- Aggregate functions

are **read-only** in MySQL. They are used purely for reporting.

In [ ]:
CREATE OR REPLACE VIEW count_per_dept AS
SELECT deptno, COUNT(*) AS cnt
FROM emp
GROUP BY deptno;

In [ ]:
SELECT * FROM count_per_dept;

##  View Definition Metadata

MySQL stores view metadata in `INFORMATION_SCHEMA.VIEWS`.

In [ ]:
SELECT view_definition
FROM information_schema.views
WHERE table_schema = DATABASE()
  AND table_name = 'count_per_dept';

## 5. View with Subquery

### 
HR wants a reusable dataset showing employees earning **above the company average salary**.

Views can encapsulate complex logic including:
- Subqueries
- Derived tables

This improves reuse and consistency.

In [ ]:
CREATE OR REPLACE VIEW emp_above_avg_sal AS
SELECT e.ename, e.sal
FROM emp e
JOIN (
    SELECT AVG(sal) AS avg_sal
    FROM emp
) a
ON e.sal > a.avg_sal;

In [ ]:
SELECT * FROM emp_above_avg_sal;

### Inspecting View Definition

In [ ]:
SELECT view_definition
FROM information_schema.views
WHERE table_schema = DATABASE()
  AND table_name = 'emp_above_avg_sal';

## Listing All Views in the Database


MySQL exposes all view metadata through `INFORMATION_SCHEMA.VIEWS`.

In [ ]:
SELECT table_name
FROM information_schema.views
WHERE table_schema = DATABASE();

## Dependency Behavior: Dropping Base Tables

If a base table is dropped:
- Dependent views become **invalid**
- Querying them results in runtime errors

In [ ]:
-- Example (do not run in production without validation)
-- DROP TABLE emp;
-- SELECT * FROM deptno20;  -- This will fail

## Summary

- Views simplify reuse and enforce consistency
- Updatable views can accept DML under strict conditions
- Aggregate views are read-only
- `WITH CHECK OPTION` enforces data integrity
- View metadata is available in `INFORMATION_SCHEMA`
- Dropping base tables invalidates dependent views